# Assignment: MTA Arrival Clock

In the "MTA Arrival Clock" case study, we have created an alternative design to visualize the arrival time information that looks at the following:

<div><img src="https://user-images.githubusercontent.com/3606672/246465520-5b7cad68-4722-4192-ad69-d0916c8f9cea.svg" width=400/></div>

As noted in the case study, it has the following drawbacks:

* Only trains arriving within the next 10 min are displayed.
* When two or more trains have the same arrival time, the subway line symbols and corresponding annotations will overlap.

## Task:

In this assignment, your task is to come up with a new design that addresses at least one of the two aforementioned drawbacks. You can either use the existing dot plot design or come up with a complete new design. The requirements of the design is the following:

* The dimension of the visualization is fixed (833x200).
* The design should make it easy to extract the arrival time information for each of the upcoming trains.
* The arrival time information should be legible from afar.
* Be mindful about utilizing the available spaces.

Please use this notebook for this assignment.

I will be addressing trains arriving at the same time.

In [1]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "line": ["F", "F", "E", "M"],
    "arrival_time": [3, 6, 6, 0], #[3,6,6,0]
    "destination": ["Jamaica-179 St", "Kings Hwy", "Jamaica-179 St", "Forest Hills-71 Av"]})
line_color = pd.DataFrame({
    "line": ["E", "F", "M"],
    "color": ["#0039a6", "#ff6319", "#ff6319"]
})
df.sort_values(by="arrival_time", inplace=True)

In [2]:
bg_df = pd.DataFrame({
    "x": np.arange(8)
})
x_scale = alt.Scale(domain=[-1, 7.6])
background = alt.Chart(bg_df).mark_circle(size= 1000, color="gray").encode(
    x=alt.X("x:Q", axis=None, scale=x_scale),
    y=alt.value(80)
).properties(width=833, height=200)
background

alt.Chart(...)

need to work on the box thing

In [3]:
df["adjusted_arrival_time"] = df["arrival_time"] + df.groupby("arrival_time").cumcount()
df["first_wordFl"] = df["destination"].str[0]
df["display_word"] = df["line"] + "-" + df["first_wordFl"]
df

,line,arrival_time,destination,adjusted_arrival_time,first_wordFl,display_word
3,M,0,Forest Hills-71 Av,0,F,M-F
0,F,3,Jamaica-179 St,3,J,F-J
1,F,6,Kings Hwy,6,K,F-K
2,E,6,Jamaica-179 St,7,J,E-J


In [4]:
def make_unique_by_decreasing(df, column):
    while df.duplicated(subset=column).any():
        duplicates = df[df.duplicated(subset=column)]
        for idx in duplicates.index:
            df.at[idx, column] += 1
    return df

df = make_unique_by_decreasing(df, "adjusted_arrival_time")
while (df["adjusted_arrival_time"] >= 8).any():
    df["adjusted_arrival_time"] -= 1
df

,line,arrival_time,destination,adjusted_arrival_time,first_wordFl,display_word
3,M,0,Forest Hills-71 Av,0,F,M-F
0,F,3,Jamaica-179 St,3,J,F-J
1,F,6,Kings Hwy,6,K,F-K
2,E,6,Jamaica-179 St,7,J,E-J


In [5]:
arrival_time = alt.Chart(df).mark_circle(size=9500, opacity=1).encode(
    x=alt.X("adjusted_arrival_time:Q", axis=None, scale=x_scale),
    y=alt.value(80),
    color=alt.Color("color:N", scale=None)
).transform_lookup(lookup="line", from_=alt.LookupData(line_color, "line", ["color"]))
background + arrival_time 

C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\c

alt.LayerChart(...)

In [6]:
line_label = alt.Chart(df).mark_text(color="white", fontSize= 50, font="Helvetica", fontStyle="bold").encode(
    x=alt.X("adjusted_arrival_time:Q", axis=None, scale=x_scale),
    y=alt.value(80),
    text="display_word"
)
background + arrival_time + line_label

C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\c

alt.LayerChart(...)

In [7]:
dir_label = alt.Chart(df).mark_text(color="gray", fontSize=70, font="Helvetica", fontStyle="bold")\
    .encode(x=alt.value(20), y=alt.value(80), text=alt.value("❮"))
mins_label = alt.Chart(df).mark_text(color="black", fontSize=25, font="Helvetica").encode(x=alt.value(35), y=alt.value(170),
                                                                                         text=alt.value("MINS:"))
background + dir_label + mins_label + arrival_time + line_label

C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\c

alt.LayerChart(...)

In [8]:
time_label = alt.Chart(df).mark_text(color="black", fontSize=70, font="Helvetica").encode(
    x=alt.X("adjusted_arrival_time:Q", axis=None, scale=x_scale),
    y=alt.value(170),
    text="arrival_time")
background + dir_label + mins_label + arrival_time + line_label + time_label

C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\c

alt.LayerChart(...)

In [9]:
destination_label = alt.Chart(df)\
.mark_text(color="black", fontSize=12, font="Helvetica").encode(
    x=alt.X("adjusted_arrival_time:Q", axis=None, scale=x_scale),
    y=alt.value(20),
    text="destination:N"
)
background + dir_label + mins_label + arrival_time + line_label + time_label + destination_label

C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\gto219\Anaconda3\lib\site-packages\altair\utils\c

alt.LayerChart(...)

Code above can show up to 7 mins arrival time. The decreased available spacing allows for enlarged images and time displays. 
Trains that are arriving at the same time will be next to each other and display the same time underneath.
The direction 